In [ ]:
# Generic imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.metrics import (
    confusion_matrix,
    cohen_kappa_score
)
from tqdm import tqdm   # For keeping track of loops

In [ ]:
# Custom imports
from src.diagnosis_tools import (
    mark_hypoxemic_episodes,
    mark_abnormal_cxr,
    mark_cxr_within_48h_of_post_vent_hypoxemia,
    mark_note_within_7d,
    mark_notes_with_ml,
    text_match_risk_factors,
    diagnose_or_exclude_encounters,
    flag_echos
)
import src.plots as plots

In [ ]:
# set plotting params
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
plt.style.reload_library()
rcparams = plots.stdrcparams1()
mpl.rcParams.update(rcparams)

In [ ]:
# Custom display of tables for easier inspection
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

While criteria to identify ARDS includes more than these, from the perspective of files these will be the important files/criteria:  
-PF ratios (ARDS if PF<=300 mmHg and PEEP>=5 cm H20 anytime during encounter)  
-Chest X-ray reports (ARDS if bilateral pulmonary opacities identified within 48 h window of PF ratio <= 300).  
-First and second criteria happening within 7 days of a known ARDS risk factor (-1 to 7 days of latest timestamp of above combo). Search attending physician notes for this.  
-If no risk factor found, seek language ruling out cardiac failure in attending physician notes.  
-If no cardiac failure language found, use objective criteria form echocardiography reports to rule out cardiac failure, otherwise, ARDS is adjudicated (window of entire hospitalization).

## Read in the tables

In [ ]:
basedir = Path("..")
training_location = basedir / 'Analysis_Data' / 'train_ML'
path = basedir / 'Analysis_Data' / 'MIMIC_III' / 'labeled_subset'
raw_path = basedir / 'Raw_Data' / 'MIMIC_III' / 'labeled_subset'
characteristics_path = basedir / 'Raw_Data' / 'MIMIC_III' / 'mimic_characteristics'
figure_path = basedir / 'Figures'

In [ ]:
pf = pd.read_csv(path / "pf_ratio.csv")
pf['pf_ratio_timestamp'] = pd.to_datetime(pf['pf_ratio_timestamp'])
pf['vent_start_timestamp'] = pd.to_datetime(pf['vent_start_timestamp'])

try:
    peep = pd.read_csv(path / "peep.csv")
    peep['peep_timestamp'] = pd.to_datetime(peep['peep_timestamp'])
    
except FileNotFoundError:
    peep = None
    print("This dataset doesn't seem to have peep separately specified.")

cxr = pd.read_csv(path / "cxr.csv")
cxr['cxr_timestamp'] = pd.to_datetime(cxr['cxr_timestamp'])

notes = pd.read_csv(path / "attending_notes.csv")
notes['notes_timestamp'] = pd.to_datetime(notes['notes_timestamp'])

echo = pd.read_csv(path / "echo_reports.csv")
echo['echo_timestamp'] = pd.to_datetime(echo['echo_timestamp'])

bnp = pd.read_csv(path / "bnp.csv")
bnp['bnp_timestamp'] = pd.to_datetime(bnp['bnp_timestamp'])

final_numbers = pd.read_excel(raw_path / "ARDS_Criteria_Curt.xlsx")
final_numbers_eryn = pd.read_excel(raw_path / "ARDS_Criteria_Eryn.xlsx")

In [ ]:
pf_encounters = pf['encounter_id'].drop_duplicates()

if peep is not None:
    peep_encounters = peep['encounter_id'].drop_duplicates()
else:
    peep_encounters = None
    
cxr_encounters = cxr['encounter_id'].drop_duplicates()
notes_encounters = notes['encounter_id'].drop_duplicates()
echo_encounters = echo['encounter_id'].drop_duplicates()
bnp_encounters = bnp['encounter_id'].drop_duplicates()

if peep is None:
    total_encounters = pd.merge(pf_encounters, cxr_encounters, how='outer').drop_duplicates()
    print(f"Patient encounters with PF ratios or CXRs: {len(total_encounters)}")
else:
    total_encounters = pd.merge(pf_encounters, peep_encounters, how='outer').drop_duplicates()
    total_encounters = pd.merge(total_encounters, cxr_encounters, how='outer').drop_duplicates()
    print(f"Patient encounters with PF ratios, PEEP, or CXRs: {len(total_encounters)}")
    
total_encounters = pd.merge(total_encounters, notes_encounters, how='outer').drop_duplicates()
total_encounters = pd.merge(total_encounters, echo_encounters, how='outer').drop_duplicates()
total_encounters = pd.merge(total_encounters, bnp_encounters, how='outer').drop_duplicates()

#### This is to get the ARDS recognition rate in the MIMIC-III subset

In [ ]:
ards_recognized = final_numbers[[
    'HADM_ID',
    'ARDS DOCUMENTED IN NOTE (1=yes)'
    ]].drop_duplicates() \
        .groupby('HADM_ID')['ARDS DOCUMENTED IN NOTE (1=yes)'] \
        .sum().to_frame().reset_index()

In [ ]:
ards_by_Curt = final_numbers[[
    'HADM_ID',
    'FINAL ARDS CURT (1=YES)'
    ]].drop_duplicates() \
        .groupby('HADM_ID')['FINAL ARDS CURT (1=YES)'] \
        .sum().to_frame().reset_index()

In [ ]:
recognition_comparison = pd.merge(ards_recognized, ards_by_Curt)

In [ ]:
recognition_comparison.loc[
    recognition_comparison['FINAL ARDS CURT (1=YES)'] == 1,
    'ARDS DOCUMENTED IN NOTE (1=yes)'
    ].value_counts()

In [ ]:
g = recognition_comparison['FINAL ARDS CURT (1=YES)'] == 1
g.sum()

## Now, diagnosis.

### PF ratio table: Flagging hypoxemic windows (will check for PEEP >= 5 cm H2O if PEEP available)

In [ ]:
pf, hypox_df = mark_hypoxemic_episodes(pf, peep, 'encounter_id')

In [ ]:
print(f"Encounters with hypoxemia: {hypox_df['encounter_id'].nunique()}")
print(f"Uniquely-identified hypoxemic entries: {len(hypox_df)}")

In [ ]:
# Ground truth
peep_yes = final_numbers['Meet PEEP>=5 Criteria? (1=yes)'] == 1
hypoxemia_yes = final_numbers['Meet PF<300 Criteria? (1=yes)'] == 1

hypox_df_label = final_numbers.loc[peep_yes & hypoxemia_yes].drop_duplicates()

In [ ]:
print(f"Encounters with hypoxemia by Curt: {hypox_df_label['HADM_ID'].nunique()}")

### Chest X-ray: Flagging abnormal CXRs and whether they are within 48 h of a hypoxemic record

#### Flagging those CXR that are "abnormal" (bilateral pulmonary opacities consistent with pulmonary edema).

In [ ]:
train_data = training_location / 'cxr_whole_training_dataset.csv'

cxr = mark_abnormal_cxr(
    cxr,
    train_data,
    train_col=['segmented_report', 'score'],
    test_label_col='curt_bl_infiltrates_(1=yes)',
    thresholding="default"
    )

#### Flagging CXRs that are within 48 h of a hypoxemic entry

In [ ]:
cxr, hypox_pred_abn_cxr_48h = mark_cxr_within_48h_of_post_vent_hypoxemia(
    hypox_df,
    cxr,
    'encounter_id',
    'cxr_timestamp'
    )

In [ ]:
# print(f"Abnormal CXRs - label: {cxr['curt_bl_infiltrates_(1=yes)'].sum()}")
print(f"Predicted Abnormal CXRs: {cxr['cxr_score_predicted'].sum()}")

In [ ]:
# print(
#     f"""
#     Encounters with abnormal CXRs by Curt: {cxr.loc[
#         cxr['curt_bl_infiltrates_(1=yes)'].astype(bool),
#         'encounter_id'].nunique()}
#     """
#     )

print(
    f"""Encounters with predicted abnormal CXRs: {cxr.loc[
        cxr['cxr_score_predicted'],
        'encounter_id'
        ].nunique()}"""
    )

In [ ]:
print(
    f"""Encounters with hypoxemia and CXRs within 48h: {cxr.loc[
        cxr['within_48h'],
        'encounter_id'
        ].nunique()}"""
    )

print(f"Uniquely-identified entries: {len(cxr.loc[cxr['within_48h']])}")

In [ ]:
print(
    f"""Encounters with hypoxemia and predicted abnormal CXRs within 48h: {hypox_pred_abn_cxr_48h[
        'encounter_id'
        ].nunique()}"""
    )

print(f"Uniquely-identified entries: {len(hypox_pred_abn_cxr_48h)}")

In [ ]:
# Ground truth!
f = hypox_df_label['BL-PF within 48h Criteria? (1=yes)'] == 1
hypox_abn_cxr_48h = hypox_df_label.loc[f]

print(
    f"""Encounters with hypoxemia and abnormal CXRs within 48h by Curt: {hypox_abn_cxr_48h[
        'HADM_ID'
        ].nunique()}"""
    )

#### DETOUR start  
Disagreement graph

In [ ]:
within_48 = cxr.loc[cxr['within_48h']]
outside_48 = cxr.loc[~cxr['within_48h']]

In [ ]:
l1, l2 = 0.1, 0.9

In [ ]:
agg_disagree = []
r = cxr['cxr_score_probability'] < l1
s = cxr['cxr_score_probability'] >= l1
t = cxr['cxr_score_probability'] < l2
u = cxr['cxr_score_probability'] >= l2


for z in tqdm(range(100)):
    lows = cxr.loc[r].sample(n=r.sum(), replace=True, axis=0)
    intermediates = cxr.loc[s&t].sample(n=(s&t).sum(), replace=True, axis=0)
    highs = cxr.loc[u].sample(n=u.sum(), replace=True, axis=0)
    boot_within_48 = pd.concat([lows, intermediates, highs], ignore_index=True)
    
    
    a1 = boot_within_48['cxr_score_probability'] < l1
    temp = boot_within_48[a1].dropna(subset=['eryn_bl_infiltrates_(1=yes)', 'curt_bl_infiltrates_(1=yes)', 'seg_cxr_text'])
    disagreements_curt = 0
    disagreements_eryn = 0
    
    for idx, row in temp.iterrows():
        disagreements_curt += int(row["cxr_score_predicted"] != row["curt_bl_infiltrates_(1=yes)"])
        disagreements_eryn += int(row["cxr_score_predicted"] != row["eryn_bl_infiltrates_(1=yes)"])
    agg_disagree.append({'model_confidence': 'High\nconfidence\nNo', 'disagreement': disagreements_curt/len(temp), 'physician': 'Intensivist'})
    agg_disagree.append({'model_confidence': 'High\nconfidence\nNo', 'disagreement': disagreements_eryn/len(temp), 'physician': 'Internist'})


    a3 = boot_within_48['cxr_score_probability'] >= l1
    a4 = boot_within_48['cxr_score_probability'] < l2
    temp = boot_within_48[a3&a4].dropna(subset=['eryn_bl_infiltrates_(1=yes)', 'curt_bl_infiltrates_(1=yes)', 'seg_cxr_text'])
    disagreements_curt = 0
    disagreements_eryn = 0
    
    for idx, row in temp.iterrows():
        disagreements_curt += int(row["cxr_score_predicted"] != row["curt_bl_infiltrates_(1=yes)"])
        disagreements_eryn += int(row["cxr_score_predicted"] != row["eryn_bl_infiltrates_(1=yes)"])
    agg_disagree.append({'model_confidence': "Low confidence", 'disagreement': disagreements_curt/len(temp),'physician': 'Intensivist'})
    agg_disagree.append({'model_confidence': 'Low confidence', 'disagreement': disagreements_eryn/len(temp), 'physician': 'Internist'})


    a7 = boot_within_48['cxr_score_probability'] >= l2
    temp = boot_within_48[a7].dropna(subset=['eryn_bl_infiltrates_(1=yes)', 'curt_bl_infiltrates_(1=yes)', 'seg_cxr_text'])
    disagreements_curt = 0
    disagreements_eryn = 0
    
    for idx, row in temp.iterrows():
        disagreements_curt += int(row["cxr_score_predicted"] != row["curt_bl_infiltrates_(1=yes)"])
        disagreements_eryn += int(row["cxr_score_predicted"] != row["eryn_bl_infiltrates_(1=yes)"])
    agg_disagree.append({'model_confidence': "High\nconfidence\nYes", 'disagreement': disagreements_curt/len(temp), 'physician': 'Intensivist'})
    agg_disagree.append({'model_confidence': 'High\nconfidence\nYes', 'disagreement': disagreements_eryn/len(temp), 'physician': 'Internist'})
    
disagreement_df = pd.DataFrame(agg_disagree)

In [ ]:
r.sum(), (s&t).sum(), u.sum()

In [ ]:
fig1, ax1 = plt.subplots(1, 1, figsize=plots.stdfigsize(62, n_rows=1, n_cols=1, layout="single"))

sns.pointplot(data=disagreement_df, x='model_confidence', y='disagreement', hue='physician',
              dodge=0.4, errorbar=("ci", 95), capsize=0.1, ax=ax1, color="0", legend=False,
              linestyle="none")
sns.swarmplot(data=disagreement_df, x='model_confidence', y='disagreement', hue='physician',
              dodge=True, palette=["tab:blue", "tab:orange"], ax=ax1, alpha=0.65, size=2.1)

ax1.set_ylabel("Fraction disagreed")
ax1.grid(linestyle=':', axis='y')
ax1.legend(loc='upper left', frameon=False)
ax1.set_xlabel("")
ax1.set_ylim(0, 0.4)

fig1.tight_layout()
# plt.savefig(figure_path / 'SIfig8.png')
plt.show()

#### DETOUR end

### Attending physician notes

#### Flag notes within -1 to 7 days of latest of hypoxemia or abnormal CXR report.

In [ ]:
notes = mark_note_within_7d(
    notes,
    hypox_df,
    hypox_pred_abn_cxr_48h,
    'encounter_id',
    'cxr_timestamp'
    )

In [ ]:
print(
    f"""Encounters with notes within 7 days of hypox_abn_cxr: {notes.loc[
        notes['within_7d'],
        'encounter_id'
        ].nunique()}"""
        )

print(f"Uniquely-identified entries: {len(notes.loc[notes['within_7d']])}")

#### Flag notes mentioning any risk factor. Separately, flag notes with cardiac failure language.

In [ ]:
notes = mark_notes_with_ml(
    notes,
    training_location,
    train_col=['seg_pneumonia', 'pneumonia_sw'],
    test_label_col='curt_pneumonia_(1=yes)',
    thresholding="default"
    )

In [ ]:
notes = text_match_risk_factors(notes)

In [ ]:
notes, diagnosed, excluded, for_objective_assessment = diagnose_or_exclude_encounters(
    notes,
    hypox_pred_abn_cxr_48h,
    'encounter_id'
    )

In [ ]:
print(
    f"{notes.encounter_id.nunique()}, {diagnosed.encounter_id.nunique()}, {excluded.encounter_id.nunique()}, {for_objective_assessment.encounter_id.nunique()}"
    )

### BNP and ECHO reports: Objective assessment of cardiac failure

#### First, let's annotate the ECHO reports with the values/statements of interest:  
- lvef < 40%  
- cardiopulmonary bypass  
- left atrial dimension > 4 cm or volume index > 28 mL/m2  
- left ventricular hypertrophy  
- Grade II or III diastolic dysfunction

In [ ]:
# These will be dictionaries whose keys will become the column names for the flags
# and the lists will be the regex patterns to search for

# (?i) is to inactivate case-sensitivity
# (?:) is to indicate that contents inside a parenthesis shouldn't be read as a "capturing group"
# Default behavior of () is to consider it a capturing group
echo_prefix = {'lvef': ['(?i)lv\s+ejection\s+fraction',
                        '(?i)left\s+ventricular\s+ejection\s+fraction',
                        '(?i)lvef',
                        '(?i)left\s+ventricular\s+ef',
                        '(?i)lvef\s+is',
                        '(?i)left\s+ventricle\s+ejection\s+fraction\s+is',
                        '(?i)lv\s+ejection\s+fraction\s+is'],
               
               # Match "cardiopulmonary bypass" ensuring at least one whitespace character between those words
              'cp_bypass': ['(?i)cardiopulmonary\s+bypass'],
              
              'la_dimension': ['(?i)la\s+diameter',
                               '(?i)la\s+dimension'],

              'la_volume_index': ['(?i)la\s+volume',
                                  '(?i)LA\s+Vol\s+BP\s+A/L\s+Index'],
              
              'lv_hypertrophy': ['(?i)(?:left\s+ventricular|lv|lv\s+concentric)\s*hypertrophy',
                                 '(?i)LVH'],
              
              'diastolic_dysfunction': ['(?i)(grade\s*ii)',
                                        '(?i)(grade\s*iii)']}

echo_suffix = {'lvef': '\D{0,20}(\d{1,3}|\d{1,2}\s*-\s*\d{1,3})-{0,1}\s*%', # Sample matches: 45%, 45 %, 45-55%, 45 - 55 %, 45- 100%, 45- %
               'cp_bypass': '(?!\s*N\/A|\s*Patient\s+was\s+not\s+placed\s+on\s+cardiopulmonary\s+bypass|\s*NA)',  # Don't match if N/A or Patient wasn't placed on CPB
               'la_dimension': '\D{0,25}(\d\.\s*\d)\s*(?:cm|centimeter)', # Sample matches: 2.7cm, 2.7 cm, 2.7   centimeter
               
                # Match anything until "ml" appears once or never, then match anything until the number of interest appears
                # followed by either ml/m or ml per square meter
               'la_volume_index': '.*?(?:ml)?.*?(\d+\.\s*\d+)\s+(?:(?=ml\/m)|(?=ml\s+per\s+square\s+meter))',
               'lv_hypertrophy': '',
               # Matches anything, either never or up to 30 characters, then an arbitrary number of white spaces,
               # as long as "diastolic dysfunction" immediately follows.
               'diastolic_dysfunction': '.{0,30}\s*?(?=diastolic\s+dysfunction)'}

In [ ]:
echo = flag_echos(echo, echo_prefix, echo_suffix)

In [ ]:
# Encounters entering objective assessment
print(
    f"""There are {for_objective_assessment[
        'encounter_id'
        ].nunique()} unique encounters entering objective assessment""")

#### 1. Taking away encounters that have BNP > 100 pg/mL

In [ ]:
a = bnp['bnp_value'] > 100
encounters_with_bnp_greater_than_100 = list(bnp.loc[a, 'encounter_id'].unique())

j = for_objective_assessment['encounter_id'].isin(encounters_with_bnp_greater_than_100)
remaining_after_bnp = for_objective_assessment.loc[~j]

In [ ]:
# Encounters remaining after bnp exclusion
print(
    f"{len(encounters_with_bnp_greater_than_100)} encounters had BNP > 100 pg/mL. {remaining_after_bnp['encounter_id'].nunique()} encounters remain."
    )

#### 2.Taking away encounters that have left ventricular ejection fraction < 40%

In [ ]:
b = echo['lvef_value'] < 40
encounters_with_lvef_smaller_than_40 = list(echo.loc[b, 'encounter_id'].unique())

j = remaining_after_bnp['encounter_id'].isin(encounters_with_lvef_smaller_than_40)
remaining_after_lvef = remaining_after_bnp.loc[~j]

In [ ]:
# Encounters remaining after lvef exclusion
print(
    f"{len(encounters_with_lvef_smaller_than_40)} encounters had LVEF < 40%. {remaining_after_lvef['encounter_id'].nunique()} encounters remain."
    )

#### 3. Taking away encounters that had cardiopulmonary bypass in the report (as a proxy for having had cardiopulmonary bypass during the ECHO).

In [ ]:
cpb = echo['cp_bypass_value'].notnull()
encounters_with_cardiopulmonary_bypass = list(echo.loc[cpb, 'encounter_id'].unique())

j = remaining_after_lvef['encounter_id'].isin(encounters_with_cardiopulmonary_bypass)
remaining_after_cpb = remaining_after_lvef.loc[~j]

In [ ]:
# Encounters remaining after cardiopulmonary bypass exclusion
print(
    f"{len(encounters_with_cardiopulmonary_bypass)} encounters had cardiopulmonary bypass. {remaining_after_cpb['encounter_id'].nunique()} encounters remain."
    )

#### 4. Taking away encounters that have two out of three additional criteria:  
- Left atrial enlargement (either left atrial dimension > 4 cm or left atrial volume index > 28 mL/m^2)  
- Left ventricular hypertrophy  
- Grade III or Grade IV diastolic dysfunction

In [ ]:
# Scoring the presence of criteria as 0 or 1
la_dim = echo['la_dimension_value'] > 4
la_vol_idx = echo['la_volume_index_value'] > 28
echo.loc[:, 'la_enlargement_bool'] = (la_dim | la_vol_idx).astype(int)
echo.loc[:, 'lv_hypertrophy_bool'] = echo['lv_hypertrophy_value'].notnull().astype(int)
echo.loc[:, 'diastolic_dysfunction_bool'] = echo['diastolic_dysfunction_value'].notnull().astype(int)

echo['additional_criteria_count'] = echo['la_enlargement_bool'] + \
                                    echo['lv_hypertrophy_bool'] + \
                                    echo['diastolic_dysfunction_bool']

In [ ]:
add_crit = echo['additional_criteria_count'] > 1
encounters_with_additional_criteria = list(echo.loc[add_crit, 'encounter_id'].unique())

j = remaining_after_cpb['encounter_id'].isin(encounters_with_additional_criteria)
remaining_after_additional_criteria = remaining_after_cpb.loc[~j]

In [ ]:
# Encounters remaining after additional criteria exclusion
print(
      f"{len(encounters_with_additional_criteria)} encounters had two out of three additional criteria. {remaining_after_additional_criteria['encounter_id'].nunique()} encounters remain."
      )

# Diagnosed encounters

In [ ]:
diagnosed_encntrs = diagnosed['encounter_id'].nunique()

In [ ]:
text1 = "Encounters diagnosed:\n\nPF+CXR+Notes: {diagnosed_encntrs}".format(
    diagnosed_encntrs = diagnosed['encounter_id'].nunique())
text2 = "\nNo risk factor nor objective evidence of cardiac failure: {encntrs_remaining}".format(
    encntrs_remaining = remaining_after_additional_criteria['encounter_id'].nunique())
text3 = "\nTotal encounters diagnosed: {total}".format(
    total = diagnosed_encntrs+remaining_after_additional_criteria['encounter_id'].nunique())

print(text1+text2+text3)

### Massaging the encounters to get list of pipeline diagnosed vs. not diagnosed table

In [ ]:
encounters_diagnosed_by_pipeline = pd.merge(
    diagnosed['encounter_id'].drop_duplicates(),
    remaining_after_additional_criteria['encounter_id'].drop_duplicates(),
    how='outer'
    ).drop_duplicates()

In [ ]:
# Creating table with all MIMIC encounters and a diagnosed flag for each
encounter_summary = pd.merge(
    total_encounters,
    encounters_diagnosed_by_pipeline,
    how='outer',
    indicator=True
    )

encounter_summary = encounter_summary.replace(
    to_replace={
        '_merge': {
            "left_only": 'No',
            "both": 'Yes'
            }
        }
    )

encounter_summary = encounter_summary.rename(
    columns={
        '_merge': "pipeline_diagnosed"
        }
    )

### Now, massaging the labels table

In [ ]:
# This is Curt's table: Changing column names to match the other tables
final_numbers.rename(
    columns={
        'HADM_ID': 'encounter_id',
        'FINAL ARDS CURT (1=YES)': 'curt_diagnosed'
        },
    inplace=True
    )

final_numbers_eryn.rename(
    columns={
        'HADM_ID ERYN': 'encounter_id',
        'FINAL ARDS ERYN (1=YES)': 'eryn_diagnosed'
        },
    inplace=True
    )

In [ ]:
# The final_numbers table contains a bunch of other stuff.
# This subsets the table for the two columns we care about in this analysis
# And then takes care of instances where an encounter might have two different diagnoses:
# If it was ever ARDS, consider it ARDS.
encounters_by_Curt = final_numbers[[
    'encounter_id',
    'curt_diagnosed']] \
        .drop_duplicates() \
        .groupby('encounter_id')['curt_diagnosed'] \
        .sum().to_frame().reset_index()   
        
encounters_by_Eryn = final_numbers_eryn[[
    'encounter_id',
    'eryn_diagnosed']] \
        .drop_duplicates() \
        .groupby('encounter_id')['eryn_diagnosed'] \
        .sum().to_frame().reset_index()

### Merging adjudications

In [ ]:
# Adding encounters diagnosed by Curt
pipeline_v_curt = pd.merge(encounter_summary, encounters_by_Curt, how='outer')

pipeline_v_curt = pipeline_v_curt.replace(
    to_replace={
        'curt_diagnosed': {0: "No", 1: "Yes"}
        }
    )

In [ ]:
# Adding encounters diagnosed by Eryn
eryn_v_curt = pd.merge(encounters_by_Curt, encounters_by_Eryn, how='outer')

eryn_v_curt = eryn_v_curt.replace(
    to_replace={
        'curt_diagnosed': {0: "No", 1: "Yes"},
        'eryn_diagnosed': {0: "No", 1: "Yes"}
        }
    )

### Plot

In [ ]:
y_true = pipeline_v_curt['curt_diagnosed']
y_pred = pipeline_v_curt['pipeline_diagnosed']
cf = confusion_matrix(y_true, y_pred).transpose()[::-1, ::-1]

strings = np.asarray([['True positives\n', 'False positives\n'],
                      ['False negatives\n', 'True negatives\n']])

labels = (np.asarray(["{0} {1:.0f}".format(string, value)
                      for string, value in zip(strings.flatten(),
                                               cf.flatten())])
          ).reshape(2, 2)

In [ ]:
y_true1 = eryn_v_curt['curt_diagnosed']
y_pred1 = eryn_v_curt['eryn_diagnosed']
cf1 = confusion_matrix(y_true1, y_pred1).transpose()[::-1, ::-1]

labels1 = (np.asarray(["{0} {1:.0f}".format(string, value)
                      for string, value in zip(strings.flatten(),
                                               cf1.flatten())])
           ).reshape(2, 2)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=plots.stdfigsize(0, n_rows=2, layout='single'))

sns.heatmap(cf, fmt='', annot=labels, cmap='Blues', cbar=False, ax=ax[0])
ax[0].set_ylabel("Pipeline adjudicated")
ax[0].set_xlabel("Ground truth")
ax[0].tick_params(axis='both', bottom=False, left=False,
                  labelbottom=False, labelleft=False)
ax[0].text(-0.06, 1.05, "b", transform=ax[0].transAxes,
           fontweight='bold', va='top')

sns.heatmap(cf1, fmt='', annot=labels1, cmap='Blues', cbar=False, ax=ax[1])
ax[1].set_ylabel("Less-experienced physician adjudicated")
ax[1].set_xlabel("Ground truth")
ax[1].tick_params(axis='both', bottom=False, left=False,
                  labelbottom=False, labelleft=False)

plt.tight_layout()
# plt.savefig(figure_path / 'fig8b_diagnosis_mimic_default.png')
# plt.savefig(figure_path / 'fig8b_diagnosis_mimic_default.pdf')
plt.show()

## Clinical characteristics of ARDS patients

In [ ]:
age_los_mortality = pd.read_csv(characteristics_path / 'age_los_mortality.csv')[
    ['HADM_ID', 'age', 'LOS', 'HOSPITAL_EXPIRE_FLAG']
    ].drop_duplicates()

gender = pd.read_csv(characteristics_path / 'gender.csv')[
    ['HADM_ID', 'GENDER']
    ].drop_duplicates()

height = pd.read_csv(characteristics_path / 'height.csv')[
    ['HADM_ID', 'CHARTTIME', 'height', 'unit']
    ].drop_duplicates()

plateau_pressure = pd.read_csv(characteristics_path / 'plateau_pressure.csv')[
    ['HADM_ID', 'CHARTTIME', 'plateau_pressure', 'unit']
    ].drop_duplicates()

tidal_volume = pd.read_csv(characteristics_path / 'tidal_volume.csv')[
    ['HADM_ID', 'CHARTTIME', 'tidal_volume', 'unit', 'LABEL']
    ].drop_duplicates()

In [ ]:
# Subsetting by encounters that actually went through the pipeline
age_los_mortality = age_los_mortality.loc[age_los_mortality['HADM_ID'].isin(pipeline_v_curt['encounter_id'])]
gender = gender.loc[gender['HADM_ID'].isin(pipeline_v_curt['encounter_id'])]
height = height.loc[height['HADM_ID'].isin(pipeline_v_curt['encounter_id'])]
plateau_pressure = plateau_pressure.loc[plateau_pressure['HADM_ID'].isin(pipeline_v_curt['encounter_id'])]
tidal_volume = tidal_volume.loc[tidal_volume['HADM_ID'].isin(pipeline_v_curt['encounter_id'])]

In [ ]:
# Flagging which encounters were diagnosed with ARDS by whom
pipeline_diagnosed = age_los_mortality['HADM_ID'].isin(pipeline_v_curt.loc[pipeline_v_curt['pipeline_diagnosed'] == 'Yes', 'encounter_id'])
age_los_mortality['pipeline_diagnosed'] = pipeline_diagnosed

curt_diagnosed = age_los_mortality['HADM_ID'].isin(pipeline_v_curt.loc[pipeline_v_curt['curt_diagnosed'] == 'Yes', 'encounter_id'])
age_los_mortality['curt_diagnosed'] = curt_diagnosed


pipeline_diagnosed = tidal_volume['HADM_ID'].isin(pipeline_v_curt.loc[pipeline_v_curt['pipeline_diagnosed'] == 'Yes', 'encounter_id'])
tidal_volume['pipeline_diagnosed'] = pipeline_diagnosed

curt_diagnosed = tidal_volume['HADM_ID'].isin(pipeline_v_curt.loc[pipeline_v_curt['curt_diagnosed'] == 'Yes', 'encounter_id'])
tidal_volume['curt_diagnosed'] = curt_diagnosed


pipeline_diagnosed = plateau_pressure['HADM_ID'].isin(pipeline_v_curt.loc[pipeline_v_curt['pipeline_diagnosed'] == 'Yes', 'encounter_id'])
plateau_pressure['pipeline_diagnosed'] = pipeline_diagnosed

curt_diagnosed = plateau_pressure['HADM_ID'].isin(pipeline_v_curt.loc[pipeline_v_curt['curt_diagnosed'] == 'Yes', 'encounter_id'])
plateau_pressure['curt_diagnosed'] = curt_diagnosed


pipeline_diagnosed = pf['encounter_id'].isin(pipeline_v_curt.loc[pipeline_v_curt['pipeline_diagnosed'] == 'Yes', 'encounter_id'])
curt_diagnosed = pf['encounter_id'].isin(pipeline_v_curt.loc[pipeline_v_curt['curt_diagnosed'] == 'Yes', 'encounter_id'])
pf['curt_diagnosed'] = curt_diagnosed
pf['pipeline_diagnosed'] = pipeline_diagnosed

### Age

In [ ]:
# Sometimes patients have a birthday while in the hospital
# So they were one age on their first ICU stay, and another in subsequent stays
# Taking the age at the first ICU stay
for_age = age_los_mortality[
    ['HADM_ID', 'age', 'pipeline_diagnosed', 'curt_diagnosed']
    ].drop_duplicates().sort_values(by=['HADM_ID', 'age']).drop_duplicates(subset='HADM_ID', keep='first')

In [ ]:
for_age.loc[for_age['pipeline_diagnosed'], 'age'].describe(), for_age.loc[for_age['curt_diagnosed'], 'age'].describe()

In [ ]:
for_age.loc[~for_age['pipeline_diagnosed'], 'age'].describe(), for_age.loc[~for_age['curt_diagnosed'], 'age'].describe()

### Length of stay

In [ ]:
for_los = age_los_mortality[
    ['HADM_ID', 'LOS', 'pipeline_diagnosed', 'curt_diagnosed']
    ].drop_duplicates()

In [ ]:
for_los.loc[for_los['pipeline_diagnosed'], 'LOS'].describe(), for_los.loc[for_los['curt_diagnosed'], 'LOS'].describe()

In [ ]:
for_los.loc[~for_los['pipeline_diagnosed'], 'LOS'].describe(), for_los.loc[~for_los['curt_diagnosed'], 'LOS'].describe()

### Mortality

In [ ]:
for_mortality = age_los_mortality[
    ['HADM_ID', 'HOSPITAL_EXPIRE_FLAG', 'pipeline_diagnosed', 'curt_diagnosed']
    ].drop_duplicates()

In [ ]:
# Numerator
for_mortality.loc[
    ~for_mortality['pipeline_diagnosed'],
    ['HADM_ID', 'HOSPITAL_EXPIRE_FLAG']].drop_duplicates()['HOSPITAL_EXPIRE_FLAG'].sum()

In [ ]:
# Denominator
for_mortality.loc[
    ~for_mortality['pipeline_diagnosed'],
    ['HADM_ID', 'HOSPITAL_EXPIRE_FLAG']].drop_duplicates().shape[0]

In [ ]:
# Numerator
for_mortality.loc[
    ~for_mortality['curt_diagnosed'],
    ['HADM_ID', 'HOSPITAL_EXPIRE_FLAG']].drop_duplicates()['HOSPITAL_EXPIRE_FLAG'].sum()

In [ ]:
# Denominator
for_mortality.loc[
    ~for_mortality['curt_diagnosed'],
    ['HADM_ID', 'HOSPITAL_EXPIRE_FLAG']].drop_duplicates().shape[0]

### Low Tidal Volume Ventilation

In [ ]:
tidal_volume = tidal_volume.loc[tidal_volume['LABEL'] == "Tidal Volume (set)"].drop_duplicates()

In [ ]:
tidal_volume = pd.merge(tidal_volume, gender, on="HADM_ID")

In [ ]:
# Convert all entries in height column from cm to inches
height.loc[height['unit'] == 'cm', "height"] = height.loc[height['unit'] == 'cm', 'height'] / 2.54

In [ ]:
height = height.groupby(["HADM_ID", "CHARTTIME"])['height'].min().reset_index()

In [ ]:
tidal_volume = pd.merge(
    tidal_volume.drop(columns=['CHARTTIME', 'unit', 'LABEL']).drop_duplicates(),
    height[['HADM_ID', 'height']].drop_duplicates(),
    on='HADM_ID')

In [ ]:
male = tidal_volume['GENDER'] == 'M'
female = tidal_volume['GENDER'] == 'F'

tidal_volume.loc[male, 'standardized_tidal_volume'] = tidal_volume.loc[male, 'tidal_volume'] / (50 + 2.3*(tidal_volume.loc[male, 'height'] - 60))
tidal_volume.loc[female, 'standardized_tidal_volume'] = tidal_volume.loc[female, 'tidal_volume'] / (45.5 + 2.3*(tidal_volume.loc[female, 'height'] - 60))

In [ ]:
ltvv = tidal_volume['standardized_tidal_volume'] <= 6.5

In [ ]:
tidal_volume.loc[ltvv & ~tidal_volume['pipeline_diagnosed'], 'HADM_ID'].nunique(), tidal_volume.loc[~tidal_volume['pipeline_diagnosed'], 'HADM_ID'].nunique()

In [ ]:
tidal_volume.loc[ltvv & ~tidal_volume['curt_diagnosed'], 'HADM_ID'].nunique(), tidal_volume.loc[~tidal_volume['curt_diagnosed'], 'HADM_ID'].nunique()

### Plateau pressure

In [ ]:
plateau_proper = plateau_pressure['plateau_pressure'] > 30
plateau_pressure.loc[plateau_proper & plateau_pressure['pipeline_diagnosed'], 'HADM_ID'].nunique(), plateau_pressure.loc[plateau_pressure['pipeline_diagnosed'], 'HADM_ID'].nunique()

In [ ]:
plateau_pressure.loc[plateau_proper & plateau_pressure['curt_diagnosed'], 'HADM_ID'].nunique(), plateau_pressure.loc[plateau_pressure['curt_diagnosed'], 'HADM_ID'].nunique()

### PF ratio

In [ ]:
pf_min = pf.groupby(['encounter_id', 'pipeline_diagnosed', 'curt_diagnosed'])['pf_ratio_value'].min().reset_index()

In [ ]:
pf.loc[pipeline_diagnosed, 'pf_ratio_value'].describe(), pf.loc[curt_diagnosed, 'pf_ratio_value'].describe()

In [ ]:
pf.loc[~pipeline_diagnosed, 'pf_ratio_value'].describe(), pf.loc[~curt_diagnosed, 'pf_ratio_value'].describe()